Notes:
So does training our ratings on the Kuala Lumpur Major and Chongqing Major, plus some minors and DreamLeague seasons, and using the MDL Disneyland Paris Major as a test set sound good?
https://liquipedia.net/dota2/Dota_Pro_Circuit/2018-19
Then as a final comment we can make some predictions on the future EPICENTER Major

Most of them appear in at least 2 tournaments, but there are some in the first 2 majors not in the Disneyland one... We could do the pro circuit for 2017-18 instead and use the International 2018 as our test set
Let me take a look at how viable that is
https://liquipedia.net/dota2/Dota_Pro_Circuit/2017-18
https://liquipedia.net/dota2/The_International/2018
I think it should be enough? We can pick and choose tournaments from the circuit based on whether the teams were in the International
I downloaded TrueSkill off of github and I'm having trouble figuring it out atm


### Imports and TrueSkill set up

In [188]:
import trueskill
import json
import urllib2
from math import sqrt
trueskill.TrueSkill(draw_probability=0, backend='scipy')

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%, backend='scipy')

### Creating Players and Teams

In [189]:
# Team Indexes
team_names = [
    'Virtus.Pro', 
    'Team Liquid', 
    'PSG.LGD', 
    'Team Secret', 
    'Mineski', 
    'Vici Gaming',
    'Newbee',
    'VGJ.Thunder',
    'OG', 
    'Winstrike Team',
    'Team Serenity',
    'Invictus Gaming2',
    'Fnatic',
    'TNC Predator',
    'VGJ.Storm',
    'Evil Geniuses',
    'OpTic Gaming',
    'paiN Gaming',
    'Natus Vincere',
    'FlyToMoon',
    'LGD.Forever Young',
    'Complexity Gaming',
    'Going In',
    'Immortals',
    'Team Kinguin',
    'Vega Squadron',
    'Keen Gaming',
    'Echo International',
    'SG e-sports',
    'Digital Chaos',
    'The Final Tribe',
    'Rex Regum QEON',
    'Infamous',
    'Team Empire',
    'Invictus Gaming',
    'LGD Gaming',
    'iG Vitality',
    'Team Spirit',
]

team_index = {}

for index, name in enumerate(team_names):
    team_index[name] = index

print team_index

# Creating teams
teams = [[Rating() for _ in range(5)] for _ in range(len(team_names))]

# Example use
# print "Newbee Team before win: "
# for index, player in enumerate(teams[team_index['Newbee']]):
#     print "Player {}: ".format(index+1), player

# teams[team_index['Newbee']], teams[team_index['VGJ.Thunder']] = trueskill.rate([teams[team_index['Newbee']], teams[team_index['VGJ.Thunder']]], ranks=[0, 1])

# print "Newbee Team after win: "
# for index, player in enumerate(teams[team_index['Newbee']]):
#     print "Player {}: ".format(index+1), player

{'Newbee': 6, 'TNC Predator': 13, 'Virtus.Pro': 0, 'Digital Chaos': 29, 'Team Empire': 33, 'Immortals': 23, 'VGJ.Storm': 14, 'Rex Regum QEON': 31, 'Team Serenity': 10, 'VGJ.Thunder': 7, 'Vega Squadron': 25, 'Echo International': 27, 'paiN Gaming': 17, 'SG e-sports': 28, 'PSG.LGD': 2, 'Vici Gaming': 5, 'The Final Tribe': 30, 'Fnatic': 12, 'Infamous': 32, 'Winstrike Team': 9, 'Team Secret': 3, 'Complexity Gaming': 21, 'Natus Vincere': 18, 'iG Vitality': 36, 'LGD Gaming': 35, 'Team Spirit': 37, 'OG': 8, 'FlyToMoon': 19, 'Going In': 22, 'OpTic Gaming': 16, 'Keen Gaming': 26, 'Invictus Gaming2': 11, 'Mineski': 4, 'Team Kinguin': 24, 'Team Liquid': 1, 'Evil Geniuses': 15, 'Invictus Gaming': 34, 'LGD.Forever Young': 20}


### Learning from Training Data

We got the training data (win/loss between team matches in playoffs) from the Dota Pro Circuit 2017-2018 games. Each section contains the name of the tournament as a comment with the match data in chronological order in the list `matches`. Then we traversed the matches to update the TrueSkill value of our teams.

In [190]:
matches = [] # [winner, loser, win_count, lose_count] # win_count is number of wins for winner, lose_count is number of wins for loser

# StarLadder i-League Invitational Season 3
matches.append( [ 'Complexity Gaming', 'Mineski', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Team Secret', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Mineski', 3, 1 ] )

# PGL Open Bucharest
matches.append( [ 'Mineski', 'Immortals', 2, 1 ] )
matches.append( [ 'PSG.LGD', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Mineski', 'PSG.LGD', 2, 0 ] )

# ESL One Hamburg 2017
matches.append( [ 'Team Secret', 'Team Liquid', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Newbee', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Team Secret', 2, 0 ] )

# AMD Sapphire Dota PIT League
matches.append( [ 'Newbee', 'SG e-sports', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Immortals', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'OG', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Fnatic', 2, 1 ] )
matches.append( [ 'SG e-sports', 'Immortals', 1, 0 ] )
matches.append( [ 'OG', 'Fnatic', 1, 0 ] )
matches.append( [ 'Team Liquid', 'Newbee', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Virtus.Pro', 'SG e-sports', 1, 0 ] )
matches.append( [ 'Newbee', 'OG', 1, 0 ] )
matches.append( [ 'Newbee', 'Virtus.Pro', 1, 0 ] )
matches.append( [ 'Team Liquid', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'Newbee', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Vici Gaming', 3, 2 ] )

# Perfect World Masters
matches.append( [ 'Team Secret', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Mineski', 2, 0 ] )
matches.append( [ 'Newbee', 'LGD.Forever Young', 2, 1 ] )
matches.append( [ 'Team Kinguin', 'Vega Squadron', 2, 1 ] )
matches.append( [ 'Complexity Gaming', 'Mineski', 1, 0 ] )
matches.append( [ 'LGD.Forever Young', 'Vega Squadron', 1, 0 ] )
matches.append( [ 'Vici Gaming', 'Team Secret', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Kinguin', 2, 0 ] )
matches.append( [ 'Complexity Gaming', 'Team Kinguin', 2, 1 ] )
matches.append( [ 'LGD.Forever Young', 'Team Secret', 2, 0 ] )
matches.append( [ 'LGD.Forever Young', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Newbee', 'Vici Gaming', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'LGD.Forever Young', 2, 0 ] )
matches.append( [ 'Newbee', 'Vici Gaming', 3, 0 ] )

# DreamLeague Season 8
matches.append( [ 'Virtus.Pro', 'Natus Vincere', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Fnatic', 2, 0 ] )
matches.append( [ 'Team Secret', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Newbee', 'Infamous', 2, 0 ] )
matches.append( [ 'Natus Vincere', 'Fnatic', 2, 1 ] )
matches.append( [ 'Evil Geniuses', 'Infamous', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Virtus.Pro', 2, 1 ] )
matches.append( [ 'Team Secret', 'Newbee', 2, 0 ] )
matches.append( [ 'Natus Vincere', 'Newbee', 2, 1 ] )
matches.append( [ 'Evil Geniuses', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Evil Geniuses', 'Natus Vincere', 2, 0 ] )
matches.append( [ 'Team Secret', 'Team Liquid', 2, 1 ] )
matches.append( [ 'Team Liquid', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Team Secret', 'Team Liquid', 3, 0 ] )

# MDL Macau
matches.append( [ 'OG', 'Natus Vincere', 2, 0 ] )
matches.append( [ 'TNC Predator', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'OG', 'TNC Predator', 2, 0 ] )

# DOTA Summit 8
matches.append( [ 'Complexity Gaming', 'OpTic Gaming', 2, 1 ] )
matches.append( [ 'OG', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'OG', 'Complexity Gaming', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Team Kinguin', 2, 0 ] )
matches.append( [ 'Fnatic', 'OG', 2, 1 ] )
matches.append( [ 'Virtus.Pro', 'Fnatic', 3, 1 ] )

# Captains Draft 4.0
matches.append( [ 'Mineski', 'paiN Gaming', 2, 0 ] )
matches.append( [ 'Team Secret', 'Evil Geniuses', 2, 1 ] )
matches.append( [ 'Vici Gaming', 'Complexity Gaming', 2, 0 ] )
matches.append( [ 'OG', 'Team Empire', 2, 0 ] )
matches.append( [ 'Team Secret', 'Mineski', 2, 0 ] )
matches.append( [ 'Vici Gaming', 'OG', 2, 0 ] )
matches.append( [ 'Team Secret', 'Vici Gaming', 3, 2 ] )

# ESL One Genting 2018
matches.append( [ 'Newbee', 'Vici Gaming', 2, 0 ] )
matches.append( [ 'Evil Geniuses', 'Virtus.Pro', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Secret', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Evil Geniuses', 2, 0 ] )
matches.append( [ 'Newbee', 'Team Liquid', 3, 2 ] )

# StarLadder i-League Invitational Season 4
matches.append( [ 'PSG.LGD', 'Newbee', 2, 0 ] )
matches.append( [ 'Team Liquid', 'Mineski', 2, 0 ] )
matches.append( [ 'Team Liquid', 'PSG.LGD', 3, 1 ] )

# ESL One Katowice
matches.append(['Team Liquid', 'Team Secret', 2, 1])
matches.append(['Virtus.Pro', 'Evil Geniuses', 2, 1])
matches.append(['Vici Gaming', 'Team Liquid', 2, 1])
matches.append(['Virtus.Pro', 'Fnatic', 2, 0])
matches.append(['Virtus.Pro', 'Vici Gaming', 3, 1])
matches.append(['Team Liquid', 'Fnatic', 2, 1])

# Bucharest Major
matches.append(['VGJ.Thunder', 'TNC Predator', 2, 1])
matches.append(['Newbee', 'Team Secret', 2, 0])
matches.append(['Team Liquid', 'Evil Geniuses', 2, 1])
matches.append(['Virtus.Pro', 'OpTic Gaming', 2, 0])
matches.append(['VGJ.Thunder', 'Newbee', 2, 0])
matches.append(['Virtus.Pro', 'Team Liquid', 2, 1])
matches.append(['Virtus.Pro', 'VGJ.Thunder', 3, 0])

# Indonesia Minor
matches.append(['Natus Vincere', 'Digital Chaos', 1, 0])
matches.append(['The Final Tribe', 'Rex Regum QEON', 1, 0])
matches.append(['Natus Vincere', 'Fnatic', 2, 1])
matches.append(['Infamous', 'The Final Tribe', 2, 1])
matches.append(['Evil Geniuses', 'Natus Vincere', 2, 0])
matches.append(['VGJ.Thunder', 'Infamous', 2, 1])
matches.append(['Evil Geniuses', 'VGJ.Thunder', 2, 0])

# DreamLeague Season 9
matches.append(['Team Liquid', 'paiN Gaming', 2, 0])
matches.append(['Fnatic', 'OG', 2, 0])
matches.append(['Newbee', 'Immortals', 2, 0])
matches.append(['Team Secret', 'Team Empire', 2, 0])
matches.append(['Team Liquid', 'Fnatic', 2, 1])
matches.append(['Team Secret', 'Newbee', 2, 0])
matches.append(['OG', 'paiN Gaming', 2, 0])
matches.append(['Team Empire', 'Immortals', 2, 1])
matches.append(['Newbee', 'OG', 2, 0])
matches.append(['Fnatic', 'Team Empire', 2, 0])
matches.append(['Fnatic', 'Newbee', 2, 1])
matches.append(['Team Secret', 'Team Liquid', 2, 0])
matches.append(['Fnatic', 'Team Liquid', 2, 0])
matches.append(['Team Secret', 'Fnatic', 3, 0])

# DotA 2 Asia Championships
matches.append(['Evil Geniuses', 'Newbee', 2, 0])
matches.append(['Team Liquid', 'VGJ.Thunder', 2, 0])
matches.append(['OpTic Gaming', 'Invictus Gaming', 2, 0])
matches.append(['Vici Gaming', 'Team Secret', 2, 0])
matches.append(['LGD Gaming', 'Evil Geniuses', 2, 0])
matches.append(['Team Liquid', 'Virtus.Pro', 2, 0])
matches.append(['Mineski', 'OpTic Gaming', 2, 0])
matches.append(['Vici Gaming', 'TNC Predator', 2, 1])
matches.append(['Virtus.Pro', 'Evil Geniuses', 1, 0])
matches.append(['TNC Predator', 'OpTic Gaming', 1, 0])
matches.append(['LGD Gaming', 'Team Liquid', 2, 1])
matches.append(['Mineski', 'Vici Gaming', 2, 1])
matches.append(['Virtus.Pro', 'Vici Gaming', 1, 0])
matches.append(['TNC Predator', 'Team Liquid', 1, 0])
matches.append(['Virtus.Pro', 'TNC Predator', 2, 1])
matches.append(['Mineski', 'LGD Gaming', 2, 0])
matches.append(['LGD Gaming', 'Virtus.Pro', 2, 1])
matches.append(['Mineski', 'LGD Gaming', 3, 2])

# StarLadder ImbaTV Invitational Season 5
matches.append(['VGJ.Thunder', 'Vega Squadron', 2, 0])
matches.append(['OpTic Gaming', 'Fnatic', 2, 1])
matches.append(['OpTic Gaming', 'VGJ.Thunder', 3, 1])

# EPICENTER XL
matches.append(['FlyToMoon', 'paiN Gaming', 1, 0])
matches.append(['Mineski', 'Team Secret', 1, 0])
matches.append(['Team Liquid', 'OG', 2, 0])
matches.append(['PSG.LGD', 'Virtus.Pro', 2, 1])
matches.append(['FlyToMoon', 'OG', 1, 0])
matches.append(['Virtus.Pro', 'Mineski', 1, 0])
matches.append(['FlyToMoon', 'Virtus.Pro', 2, 0])
matches.append(['PSG.LGD', 'Team Liquid', 2, 0])
matches.append(['Team Liquid', 'FlyToMoon', 2, 1])
matches.append(['PSG.LGD', 'Team Liquid', 3, 1])

# Thailand Minor
matches.append(['Keen Gaming', 'Evil Geniuses', 2, 1])
matches.append(['VGJ.Storm', 'The Final Tribe', 2, 1])
matches.append(['Keen Gaming', 'Team Secret', 2, 1])
matches.append(['VGJ.Storm', 'Fnatic', 2, 0])
matches.append(['VGJ.Storm', 'Keen Gaming', 2, 0])



total_number_of_matches = 0
for winner, loser, win_count, lose_count in matches:
    for _ in range(win_count):
        teams[team_index[winner]], teams[team_index[loser]] = trueskill.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[0, 1])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])
        total_number_of_matches += 1
        
    for _ in range(lose_count):
        teams[team_index[winner]], teams[team_index[loser]] = trueskill.rate([teams[team_index[winner]], teams[team_index[loser]]], ranks=[1, 0])
        teams[team_index[winner]] = list(teams[team_index[winner]])
        teams[team_index[loser]] = list(teams[team_index[loser]])
        total_number_of_matches += 1
print "Total Matches: ", total_number_of_matches

Total Matches:  348


In [191]:
teams

[[trueskill.Rating(mu=28.775, sigma=3.121),
  trueskill.Rating(mu=28.775, sigma=3.121),
  trueskill.Rating(mu=28.775, sigma=3.121),
  trueskill.Rating(mu=28.775, sigma=3.121),
  trueskill.Rating(mu=28.775, sigma=3.121)],
 [trueskill.Rating(mu=28.700, sigma=2.758),
  trueskill.Rating(mu=28.700, sigma=2.758),
  trueskill.Rating(mu=28.700, sigma=2.758),
  trueskill.Rating(mu=28.700, sigma=2.758),
  trueskill.Rating(mu=28.700, sigma=2.758)],
 [trueskill.Rating(mu=30.167, sigma=5.006),
  trueskill.Rating(mu=30.167, sigma=5.006),
  trueskill.Rating(mu=30.167, sigma=5.006),
  trueskill.Rating(mu=30.167, sigma=5.006),
  trueskill.Rating(mu=30.167, sigma=5.006)],
 [trueskill.Rating(mu=28.356, sigma=3.320),
  trueskill.Rating(mu=28.356, sigma=3.320),
  trueskill.Rating(mu=28.356, sigma=3.320),
  trueskill.Rating(mu=28.356, sigma=3.320),
  trueskill.Rating(mu=28.356, sigma=3.320)],
 [trueskill.Rating(mu=28.469, sigma=4.271),
  trueskill.Rating(mu=28.469, sigma=4.271),
  trueskill.Rating(mu=28.469

### Function to Compare WInning Chance between Two Players

In [ ]:
import trueskill as ts
from trueskill.backends import cdf
from math import sqrt

def win_chance(p1, p2): #chance p1 beats p2
    deltaMu = p1.mu - p2.mu
    rsss = sqrt(p1.sigma**2 + p2.sigma**2)
    return cdf(deltaMu/rsss)

### Predicting our Test Playoff: The International 2018

In [23]:
## Tournament: The International

# Invited teams
RAMZES666 = trueskill.Rating()
No[o]ne = trueskill.Rating()
9pasha = trueskill.Rating()
RodjER = trueskill.Rating()
Solo = trueskill.Rating()

VirtusPro = [
    RAMZES666, 
    Noone,
    _9pasha, 
    RodjER, 
    Solo,
]

 
MATUMBAMAN = trueskill.Rating()
Miracle = trueskill.Rating()
MinD_ContRoL = trueskill.Rating()
GH = trueskill.Rating()
KuroKy = trueskill.Rating()

Team_Liquid = [
    MATUMBAMAN, 
    Miracle,
    MinD_ContRoL, 
    GH, 
    KuroKy,
]

Ame = trueskill.Rating()
SomnusM = trueskill.Rating()
Chalice = trueskill.Rating()
fy = trueskill.Rating()
xNova = trueskill.Rating()

PSGLGD = [
    Ame, 
    SomnusM, 
    Chalice, 
    fy, 
    xNova, 
]

Ace = trueskill.Rating()
MidOne = trueskill.Rating()
Fata = trueskill.Rating()
YapzOr = trueskill.Rating()
Puppey = trueskill.Rating()

Team_Secret = [
    Ace, 
    MidOne, 
    Fata, 
    YapzOr, 
    Puppey,
]


Mushi = trueskill.Rating()
Moon = trueskill.Rating()
iceiceice = trueskill.Rating()
Jabz = trueskill.Rating()
ninjaboogie = trueskill.Rating()

Mineski= [
    Mushi, 
    Moon, 
    iceiceice, 
    Jabz,
    ninjaboogie, 
]

Paparazi = trueskill.Rating()
Ori = trueskill.Rating()
eLeVeN = trueskill.Rating()
LaNm = trueskill.Rating()
Fenrir = trueskill.Rating()

Vici_Gaming = [
    Paparazi,
    Ori, 
    eLeVeN, 
    LaNm, 
    Fenrir,
]

Moogy = trueskill.Rating()
Sccc = trueskill.Rating()
kpii = trueskill.Rating()
Kaka = trueskill.Rating()
Faith = trueskill.Rating()

Newbee = [
    Moogy, 
    Sccc, 
    kpii, 
    Kaka, 
    Faith,
]

Sylar = trueskill.Rating()
Freeze = trueskill.Rating()
Yang = trueskill.Rating()
Fade = trueskill.Rating()
ddc = trueskill.Rating()

VGJThunder = [
    Sylar, 
    Freeze, 
    Yang, 
    Fade, 
    ddc,
]

# Qualified Teams



### Collecting Pro Player ID from OpenDotA API

In [180]:
url = "https://api.opendota.com/api/proPlayers"
header = {'User-Agent': 'Mozilla/5.0'}
request = urllib2.Request(site,headers=header)
data = json.load(urllib2.urlopen(req))
# print data

In [179]:
pro_names = {}
for index, account in enumerate(data):
    pro_names[account['name']] = index
print pro_names
# data_teams = set()
# for index, account in enumerate(data):
#     data_teams.add(account['team_name'])
#     if account['team_name'] == 'Newbee':
#         print(account['name'])
# print data_teams


{u'Mercury': 846, u'Sedoy': 75, u'EternaLEnVy': 63, u'Van': 382, u'YapzOr': 200, u'RSG.Escanor': 848, u'Yaki': 19, u'.....': 644, u'Xm': 592, u'^Cechi': 24, u'akoadalahako': 910, u'Ohaiyo': 254, u'Creep-': 689, u'deku': 615, u'1949': 415, u'DiaMond W1nzY': 685, u'Xepher': 495, u'MagE-': 728, u'0': 703, u'343': 401, u'Shirley': 782, u'oldWhite': 414, u'Czy': 695, u'\u795e\u4e36\u5948\u5ddd': 611, u'nemphy': 353, u'BBTM': 85, u'Zitraks': 561, u'MSS-': 139, u'solen': 6, u'Biver': 385, u' .': 794, u'kenshi yonezu': 537, u'skiter': 298, u'Django': 668, u'MJW': 108, u'Babaganoush': 80, u'Puer tea': 571, u'FACEHUGGER': 780, u'Ryoya': 436, u'Exotic_Deer': 90, u'Jixing': 243, u'Ghostik': 245, u'Magical': 899, u'HoviteY': 27, u'esk': 914, u'joel': 629, u'ninjaboogie': 227, u'Harry Poter': 890, u'killomancer': 357, u'Libra': 705, u'QQQ': 209, u'\u6697\u4e2d\u89c2\u5bdf': 251, u'InJuly': 405, u'Theveryfamousar3a1234': 813, u'Farmer John': 45, u'lft.Baraban': 226, u'Panda': 406, u'Gh': 314, u'Mitch